## Login

Resume Garth

In [14]:
import garth
import os
from dotenv import load_dotenv

# Refresh dotenv
load_dotenv(override=True)

# Try to resume Garth from file, if not, login
if garth.resume("~/.garth"):
    pass
else:
    garth.login(os.getenv('EMAIL'), os.getenv('PASS'))

Save session

In [ ]:
import datetime

today = datetime.date.today()
yesterday = today - datetime.timedelta(days=1)
twoDaysAgo = today - datetime.timedelta(days=2)

print(today)
print(yesterday)
print(twoDaysAgo)

2025-06-07
2025-06-06
2025-06-05


Get Steps

In [24]:
start = twoDaysAgo
end = yesterday
garmin_connect_daily_stats_steps_url = ("/usersummary-service/stats/steps/daily")
url = f"{garmin_connect_daily_stats_steps_url}/{start}/{end}"

steps = garth.connectapi(url)

print(steps)

[{'calendarDate': '2025-06-05', 'totalSteps': 1625, 'totalDistance': 1317, 'stepGoal': 6980}, {'calendarDate': '2025-06-06', 'totalSteps': 3666, 'totalDistance': 3007, 'stepGoal': 6980}]


Stick it into a Pandas Dataframe and Clean

In [23]:
import pandas as pd

df = pd.DataFrame(steps)
df = df.dropna()

print(df)

  calendarDate  totalSteps  totalDistance  stepGoal
0   2025-06-05        1625           1317      6980
1   2025-06-06        3666           3007      6980


Connect to GSheets

In [17]:
from google.oauth2.service_account import Credentials
import gspread
from dotenv import load_dotenv

load_dotenv(override=True)

scopes = ['https://www.googleapis.com/auth/spreadsheets',
                'https://www.googleapis.com/auth/drive']


credentials = Credentials.from_service_account_file(os.getenv('SACREDS'), scopes=scopes)
gc = gspread.authorize(credentials)

Select Worksheet and Tab

In [ ]:
gs = gc.open_by_key(os.getenv('SHEETKEY'))
sheet = gs.worksheet('Steps')

<Spreadsheet 'Garmin data' id:1hCL_fghYE_xjmVv3J1tnqk-PH6WXxSpIpYWA0nzfSak>

<Worksheet 'Steps' id:0>


Append DataFrame to GSheets Tab

In [26]:
df_values = df.values.tolist()
gs.values_append('Steps', {'valueInputOption': 'RAW'}, {'values': df_values})

{'spreadsheetId': '1hCL_fghYE_xjmVv3J1tnqk-PH6WXxSpIpYWA0nzfSak',
 'tableRange': 'Steps!A1:D25',
 'updates': {'spreadsheetId': '1hCL_fghYE_xjmVv3J1tnqk-PH6WXxSpIpYWA0nzfSak',
  'updatedRange': 'Steps!A26:D27',
  'updatedRows': 2,
  'updatedColumns': 4,
  'updatedCells': 8}}